In [1]:
# Load Data
import glob 
import pandas as pd

def Carga_All_Files( ):
    regexp='../data/covi*'
    df = pd.DataFrame()
    # Iterate trough LIST DIR and 
    for my_file in glob.glob(regexp):
        this_df = pd.read_csv(my_file)
        this_df['Fecha'] = my_file
        df = pd.concat([df,this_df])
    return df  

df = Carga_All_Files( )

df.tail()


/home/usuario1/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


CCAA  Casos  Curados  Fallecidos                 Fecha  \
13      Madrid  27509   9330.0        3603  ../data/covi3103.csv   
14      Murcia    974     20.0          34  ../data/covi3103.csv   
15     Navarra   2305    192.0         113  ../data/covi3103.csv   
16  País Vasco   6320   1796.0         325  ../data/covi3103.csv   
17    La Rioja   1810    496.0          85  ../data/covi3103.csv   

    Hospitalizados      IA   ID  Nuevos     UCI  
13         15140.0  339.74  942  3419.0  1514.0  
14           283.0   58.71  952    35.0    59.0  
15          1035.0  304.49  943   159.0    99.0  
16          3594.0  251.61  950   263.0   307.0  
17           575.0  459.28  949    77.0    51.0

In [2]:
def Get_Comunidades_List( ):
    return Carga_All_Files( )['CCAA'].unique()

In [3]:
#Get_Comunidades_List()

In [4]:
def Preprocesado():
    df = Carga_All_Files( )
    # Formateamos la fecha
    df['Fecha'].replace({
        '../data/covi': '2020-',
        '.csv'        : ''}, inplace=True, regex=True)
    df['Fecha'] =  pd.to_datetime(df['Fecha'], format='%Y-%d%m')
    # 
    
    return df.sort_values(by='Fecha')

df = Preprocesado()


/home/usuario1/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [5]:
import numpy as np

def Enrich_Columns(comunidad):
    del comunidad['ID']
    del comunidad['IA']
    del comunidad['Nuevos']

    if 'Fecha' in comunidad.columns :
        comunidad.set_index('Fecha', inplace=True) 

    # Datos de fallecimientos diarios, en totales y tanto por uno.
    comunidad['Fallecidos hoy absoluto'] = comunidad['Fallecidos'] - comunidad['Fallecidos'].shift(1)
    comunidad['Fallecidos hoy porcentaje'] = comunidad['Fallecidos hoy absoluto']  / comunidad['Fallecidos'] 
    comunidad['Fallecidos hoy variacion respecto ayer'] = comunidad['Fallecidos hoy absoluto'] - comunidad['Fallecidos hoy absoluto'].shift(1)

    # Datos de Casos diarios,  en totales y tanto por uno.
    comunidad['Casos hoy absoluto'] = comunidad['Casos'] - comunidad['Casos'].shift(1)
    comunidad['Casos hoy porcentaje'] = comunidad['Casos hoy absoluto']  / comunidad['Casos'] 
    comunidad['Casos hoy variacion respecto ayer'] = comunidad['Casos hoy absoluto'] - comunidad['Casos hoy absoluto'].shift(1)

    # Convertimos a entero, para quitar decimales
    CONVERT_INT_COLUMNS = ['Fallecidos hoy absoluto', 
                           'Fallecidos hoy variacion respecto ayer',
                           'Casos hoy variacion respecto ayer',
                           'Casos hoy absoluto',
                           'Hospitalizados',
                           'Curados']
    for column in CONVERT_INT_COLUMNS :
        comunidad[column] = comunidad[column].fillna(0)
        comunidad[column] = comunidad[column].astype(np.int64)
        
    comunidad['Proporcion Curados / Casos'] = comunidad['Curados'] / comunidad['Casos'] 
    comunidad['Curados hoy absoluto']       = comunidad['Curados'] - comunidad['Curados'].shift(1)

    comunidad['Casos excluidos curados'] = comunidad['Casos'] - comunidad['Curados']  
    comunidad['Tasa Mortalidad'] = comunidad['Fallecidos'] / comunidad['Casos'] 
    

    # ordenamos las filas y columnas
    columnsTitles = ['CCAA', 
                     'Casos'     , 'Casos hoy absoluto'     , 'Casos hoy variacion respecto ayer', 'Casos hoy porcentaje'      ,
                     'Fallecidos', 'Fallecidos hoy absoluto', 'Fallecidos hoy variacion respecto ayer', 'Fallecidos hoy porcentaje' ,
                     'Tasa Mortalidad', 
                     'Curados',    'Curados hoy absoluto', 'Casos excluidos curados', 'Proporcion Curados / Casos',
                     'UCI',  
                     'Hospitalizados']
    comunidad = comunidad.reindex(columns=columnsTitles)
    comunidad = comunidad.sort_values(by=['Fecha'], ascending=False)
    comunidad = comunidad.rename(columns = {'CCAA':'Lugar'})

    return comunidad

def Get_Comunidad(nombre_comunidad):
    # Trabajamos solo con una comunidad
    df = Preprocesado()
    df = df[(df['CCAA'] == nombre_comunidad)].sort_values(by='Fecha')
    df = Enrich_Columns(df)
    return df

def Get_Nacion():
    df = Preprocesado()
    df = df.sort_values(by='Fecha')
    df = df.groupby(['Fecha']).sum()
    df['CCAA'] = 'España'
    df = Enrich_Columns(df)
    return df


In [6]:
# Just for debug purposes
def Debug_Get_Comunidad():
    comunidad = Get_Comunidad('Madrid')
    return comunidad

Debug_Get_Comunidad()



/home/usuario1/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


Lugar  Casos  Casos hoy absoluto  \
Fecha                                           
2020-04-03  Madrid  34188                2033   
2020-04-02  Madrid  32155                2315   
2020-04-01  Madrid  29840                2331   
2020-03-31  Madrid  27509                3419   
2020-03-30  Madrid  24090                1413   
2020-03-29  Madrid  22677                1157   
2020-03-28  Madrid  21520                2277   
2020-03-27  Madrid  19243                2077   
2020-03-26  Madrid  17166                2569   
2020-03-25  Madrid  14597                2245   
2020-03-24  Madrid  12352                1777   
2020-03-23  Madrid  10575                 873   
2020-03-22  Madrid   9702                 781   
2020-03-21  Madrid   8921                1756   
2020-03-20  Madrid   7165                 388   
2020-03-19  Madrid   6777                1140   
2020-03-18  Madrid   5637                 766   
2020-03-17  Madrid   4871                 706   
2020-03-16  Madrid   4165                2175   
2020-03-13  Madrid   1990                 602   
2020-03-12  Madrid   1388                 364   
2020-03-11  Madrid   1024                 242   
2020-03-10  Madrid    782                 313   
2020-03-09  Madrid    469                 332   
2020-03-06  Madrid    137                  47   
2020-03-05  Madrid     90                   0   

            Casos hoy variacion respecto ayer  Casos hoy porcentaje  \
Fecha                                                                 
2020-04-03                               -282              0.059465   
2020-04-02                                -16              0.071995   
2020-04-01                              -1088              0.078117   
2020-03-31                               2006              0.124287   
2020-03-30                                256              0.058655   
2020-03-29                              -1120              0.051021   
2020-03-28                                200              0.105809   
2020-03-27                               -492              0.107935   
2020-03-26                                324              0.149656   
2020-03-25                                468              0.153799   
2020-03-24                                904              0.143863   
2020-03-23                                 92              0.082553   
2020-03-22                               -975              0.080499   
2020-03-21                               1368              0.196839   
2020-03-20                               -752              0.054152   
2020-03-19                                374              0.168216   
2020-03-18                                 60              0.135888   
2020-03-17                              -1469              0.144939   
2020-03-16                               1573              0.522209   
2020-03-13                                238              0.302513   
2020-03-12                                122              0.262248   
2020-03-11                                -71              0.236328   
2020-03-10                                -19              0.400256   
2020-03-09                                285              0.707889   
2020-03-06                                  0              0.343066   
2020-03-05                                  0                   NaN   

            Fallecidos  Fallecidos hoy absoluto  \
Fecha                                             
2020-04-03        4483                      308   
2020-04-02        4175                      310   
2020-04-01        3865                      262   
2020-03-31        3603                      211   
2020-03-30        3392                      310   
2020-03-29        3082                      325   
2020-03-28        2757                      345   
2020-03-27        2412                      322   
2020-03-26        2090                      265   
2020-03-25        1825                      290   
2020-03-24        1535                 

In [7]:
# Just for debug purposes
def Debug_Get_Nacion():
    return Get_Nacion()

Debug_Get_Nacion()


/home/usuario1/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


Lugar   Casos  Casos hoy absoluto  \
Fecha                                            
2020-04-03  España  117710                7472   
2020-04-02  España  110238                8102   
2020-04-01  España  102136                7719   
2020-03-31  España   94417                9222   
2020-03-30  España   85195                6398   
2020-03-29  España   78797                6549   
2020-03-28  España   72248                8189   
2020-03-27  España   64059                7871   
2020-03-26  España   56188                8578   
2020-03-25  España   47610                7937   
2020-03-24  España   39673                6584   
2020-03-23  España   33089                4517   
2020-03-22  España   28572                3646   
2020-03-21  España   24926                4946   
2020-03-20  España   19980                2833   
2020-03-19  España   17147                3431   
2020-03-18  España   13716                2538   
2020-03-17  España   11178                1987   
2020-03-16  España    9191                4982   
2020-03-13  España    4209                1259   
2020-03-12  España    2950                 822   
2020-03-11  España    2128                 506   
2020-03-10  España    1622                 623   
2020-03-09  España     999                 663   
2020-03-06  España     336                  99   
2020-03-05  España     237                   0   

            Casos hoy variacion respecto ayer  Casos hoy porcentaje  \
Fecha                                                                 
2020-04-03                               -630              0.063478   
2020-04-02                                383              0.073496   
2020-04-01                              -1503              0.075576   
2020-03-31                               2824              0.097673   
2020-03-30                               -151              0.075098   
2020-03-29                              -1640              0.083112   
2020-03-28                                318              0.113346   
2020-03-27                               -707              0.122871   
2020-03-26                                641              0.152666   
2020-03-25                               1353              0.166709   
2020-03-24                               2067              0.165957   
2020-03-23                                871              0.136511   
2020-03-22                              -1300              0.127607   
2020-03-21                               2113              0.198427   
2020-03-20                               -598              0.141792   
2020-03-19                                893              0.200093   
2020-03-18                                551              0.185039   
2020-03-17                              -2995              0.177760   
2020-03-16                               3723              0.542052   
2020-03-13                                437              0.299121   
2020-03-12                                316              0.278644   
2020-03-11                               -117              0.237782   
2020-03-10                                -40              0.384094   
2020-03-09                                564              0.663664   
2020-03-06                                  0              0.294643   
2020-03-05                                  0                   NaN   

            Fallecidos  Fallecidos hoy absoluto  \
Fecha                                             
2020-04-03       10935                      932   
2020-04-02       10003                      950   
2020-04-01        9053                      864   
2020-03-31        8189                      849   
2020-03-30        7340                      812   
2020-03-29        6528                      838   
2020-03-28        5690                      832   
2020-03-27        4858                      769   
2020-03-26        4089                      655   
2020-03-25        3434                      738   
2020-03-24 